In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 4.9 MB/s 
     |████████████████████████████████| 346 kB 46.1 MB/s 
     |████████████████████████████████| 4.2 MB 61.9 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
     |████████████████████████████████| 197 kB 70.9 MB/s 
     |████████████████████████████████| 59 kB 7.5 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 212 kB 70.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 140 kB 19.0 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
     |████████████████████████████████| 271 kB 50.5 MB/s 
     |████████████████████████████████| 144 kB 58.6 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the p

In [2]:
pip install fastai

In [3]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
from torchvision import transforms
from torchvision.utils import save_image
import torch.optim as optim

is_cuda=torch.cuda.is_available()
device=torch.device("cuda" if is_cuda else "cpu")
print(device)

cuda


In [4]:
import fastai
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from fastai.vision.models.xresnet import *
from fastai.callback.mixup import *
from fastcore.script import *
from fastai.optimizer import OptimWrapper

In [5]:
path = untar_data(URLs.IMAGENETTE_160)

In [28]:
val_dl_fastai = ImageDataLoaders.from_folder(path, valid='val', 
    item_tfms=RandomResizedCrop(128, min_scale=0.35), batch_tfms=Normalize.from_stats(*imagenet_stats))
train_dl_fastai = ImageDataLoaders.from_folder(path, train='train', 
    item_tfms=RandomResizedCrop(128, min_scale=0.35), batch_tfms=Normalize.from_stats(*imagenet_stats))

In [6]:
fnames = get_image_files(path)
dblock = DataBlock()
dsets = dblock.datasets(fnames)
dblock = DataBlock(get_items = get_image_files)

lbl_dict = dict(
    n01440764='tench',
    n02102040='English springer',
    n02979186='cassette player',
    n03000684='chain saw',
    n03028079='church',
    n03394916='French horn',
    n03417042='garbage truck',
    n03425413='gas pump',
    n03445777='golf ball',
    n03888257='parachute'
)

def label_func(fname):
    return lbl_dict[parent_label(fname)]

imagenette = DataBlock(blocks = (ImageBlock, CategoryBlock),
                       get_items = get_image_files,
                       get_y = Pipeline([parent_label, lbl_dict.__getitem__]),
                       splitter = GrandparentSplitter(valid_name='val'),
                       item_tfms = RandomResizedCrop(128, min_scale=0.35),
                       batch_tfms = Normalize.from_stats(*imagenet_stats))
i_d = imagenette.datasets(path)


i_d.train
#dls = imagenette.dataloaders(path)
#dls.show_batch()

(#9469) [(PILImage mode=RGB size=213x160, TensorCategory(0)),(PILImage mode=RGB size=213x160, TensorCategory(0)),(PILImage mode=RGB size=213x160, TensorCategory(0)),(PILImage mode=RGB size=213x160, TensorCategory(0)),(PILImage mode=RGB size=160x232, TensorCategory(0)),(PILImage mode=RGB size=240x160, TensorCategory(0)),(PILImage mode=RGB size=213x160, TensorCategory(0)),(PILImage mode=RGB size=161x160, TensorCategory(0)),(PILImage mode=RGB size=240x160, TensorCategory(0)),(PILImage mode=RGB size=240x160, TensorCategory(0))...]

In [6]:
class VGG(nn.Module):
    def __init__(self):
        super().__init__()

        # Set first filter  
        self.conv1_1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        
        # He initialization: 
        nn.init.kaiming_uniform_(self.conv1_1.weight, mode='fan_in', nonlinearity='relu')
        
        self.conv1_2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        nn.init.kaiming_uniform_(self.conv1_2.weight, mode='fan_in', nonlinearity='relu')

        # Set second filter 
        self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        nn.init.kaiming_uniform_(self.conv2_1.weight, mode='fan_in', nonlinearity='relu')
        
        self.conv2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        nn.init.kaiming_uniform_(self.conv2_2.weight, mode='fan_in', nonlinearity='relu')
        
        # Set third filter
        self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        nn.init.kaiming_uniform_(self.conv3_1.weight, mode='fan_in', nonlinearity='relu')
        
        self.conv3_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        nn.init.kaiming_uniform_(self.conv3_2.weight, mode='fan_in', nonlinearity='relu')

        #nn.init.kaiming_uniform_(self.conv3_1.weight, mode='fan_in', nonlinearity='relu')
        
        self.pool = nn.MaxPool2d(2, 2)
        
        if block is 1:
          self.fc1 = nn.Linear(8192, 120)
          self.fc2 = nn.Linear(120, 10) # CHANGE 
          self.fc3 = nn.Linear(84, 10)
        else:  
          self.fc1 = nn.Linear(32768, 400)
          nn.init.kaiming_uniform_(self.fc1.weight, mode='fan_in', nonlinearity='relu')
        
          self.fc2 = nn.Linear(400, 10) # CHANGE 
          self.fc3 = nn.Linear(400, 10)

    def forward(self, x):
        if block is 1:  
          # 1 VGG block: 
          x = self.pool(F.relu(self.conv1_2(F.relu(self.conv1_1(x)))))
          x = torch.flatten(x, 1)
          x = F.relu(self.fc1(x))
          x = self.fc2(x)
        else:
          # 3 VGG block 
          x = self.pool(F.relu(self.conv1_2(F.relu(self.conv1_1(x)))))
          x = self.pool(F.relu(self.conv2_2(F.relu(self.conv2_1(x)))))
          x = self.pool(F.relu(self.conv3_2(F.relu(self.conv3_1(x)))))
          x = torch.flatten(x, 1)
          x = F.relu(self.fc1(x))
          x = self.fc3(x)

        return x

In [7]:
#|export
class BaseLoss():
    "Same as `loss_cls`, but flattens input and target."
    activation=decodes=noops
    def __init__(self, 
        loss_cls, # Uninitialized PyTorch-compatible loss
        *args,
        axis:int=-1, # Class axis
        flatten:bool=True, # Flatten `inp` and `targ` before calculating loss
        floatify:bool=False, # Convert `targ` to `float`
        is_2d:bool=True, # Whether `flatten` keeps one or two channels when applied
        **kwargs
    ):
        store_attr("axis,flatten,floatify,is_2d")
        self.func = loss_cls(*args,**kwargs)
        functools.update_wrapper(self, self.func)

    def __repr__(self) -> str: return f"FlattenedLoss of {self.func}"
    
    @property
    def reduction(self) -> str: return self.func.reduction
    
    @reduction.setter
    def reduction(self, v:str):
        "Sets the reduction style (typically 'mean', 'sum', or 'none')" 
        self.func.reduction = v

    def _contiguous(self, x:Tensor) -> TensorBase:
        "Move `self.axis` to the last dimension and ensure tensor is contigous for `Tensor` otherwise just return"
        return TensorBase(x.transpose(self.axis,-1).contiguous()) if isinstance(x,torch.Tensor) else x

    def __call__(self, 
        inp:(Tensor,list), # Predictions from a `Learner`
        targ:(Tensor,list), # Actual y label
        **kwargs
    ) -> TensorBase: # `loss_cls` calculated on `inp` and `targ`
        inp,targ  = map(self._contiguous, (inp,targ))
        if self.floatify and targ.dtype!=torch.float16: targ = targ.float()
        if targ.dtype in [torch.int8, torch.int16, torch.int32]: targ = targ.long()
        if self.flatten: inp = inp.view(-1,inp.shape[-1]) if self.is_2d else inp.view(-1)
        return self.func.__call__(inp, targ.view(-1) if self.flatten else targ, **kwargs)
    
    def to(self, device:torch.device):
        "Move the loss function to a specified `device`"
        if isinstance(self.func, nn.Module): self.func.to(device)

#|export
@delegates()
class CrossEntropyLossFlat(BaseLoss):
    "Same as `nn.CrossEntropyLoss`, but flattens input and target."
    y_int = True # y interpolation
    @use_kwargs_dict(keep=True, weight=None, ignore_index=-100, reduction='mean')
    def __init__(self, 
        *args, 
        axis:int=-1, # Class axis
        **kwargs
    ): 
        super().__init__(nn.CrossEntropyLoss, *args, axis=axis, **kwargs)
    
    def decodes(self, x:Tensor) -> Tensor:    
        "Converts model output to target format"
        return x.argmax(dim=self.axis)
    
    def activation(self, x:Tensor) -> Tensor: 
        "`nn.CrossEntropyLoss`'s fused activation function applied to model output"
        return F.softmax(x, dim=self.axis)

tst = CrossEntropyLossFlat()

In [ ]:
block=3
model = VGG() #to compile the model
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr= learning_rate,momentum=0.9) 
pytorch_adamw = partial(OptimWrapper, opt=optim.SGD(model.parameters(), lr= learning_rate,momentum=0.9) )
opt = Optimizer(params, sgd_step, lr=0.001)

criterion = nn.CrossEntropyLoss()

lbl_dict = dict(
    n01440764='tench',
    n02102040='English springer',
    n02979186='cassette player',
    n03000684='chain saw',
    n03028079='church',
    n03394916='French horn',
    n03417042='garbage truck',
    n03425413='gas pump',
    n03445777='golf ball',
    n03888257='parachute'
)


source = untar_data(URLs.IMAGENETTE_160)
fnames = get_image_files(source)

tfm = Pipeline([parent_label, lbl_dict.__getitem__, Categorize(vocab = lbl_dict.values())])
splits = GrandparentSplitter(valid_name='val')(fnames)
dsets = Datasets(fnames, [[PILImage.create], [parent_label, lbl_dict.__getitem__, Categorize]], splits=splits)
item_tfms = [ToTensor, RandomResizedCrop(128, min_scale=0.35)]
batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats)]
dls = dsets.dataloaders(after_item=item_tfms, after_batch=batch_tfms, bs=64, num_workers=8)

model = VGG() #to compile the model
model = model.to(device=device) #to send the model for training on either cuda or cpu

#tst_sgd = fastai.optimizer.OptimWrapper(model.parameters(), torch.optim.SGD, lr=1e-3, momentum=0.9, weight_decay=1e-2)
pytorch_adamw = partial(fastai.optimizer.OptimWrapper, opt=torch.optim.SGD)

learn = Learner(dls, model, metrics=accuracy,opt_func=pytorch_adamw, loss_func = tst)
learn._set_device(device)

learn.fit_one_cycle(50, 5e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.884216,1.756474,0.398217,00:30
1,1.753635,1.644160,0.454268,00:30
2,1.587111,1.526043,0.507261,00:30
3,1.502750,1.355749,0.558217,00:30
4,1.345973,1.378126,0.550828,00:30
5,1.273336,1.222539,0.590828,00:30
6,1.213820,1.100863,0.631847,00:30
7,1.129219,1.065984,0.654013,00:29
8,1.058663,1.046811,0.656560,00:30
9,1.008756,0.994588,0.678981,00:29


epoch,train_loss,valid_loss,accuracy,time
0,1.884216,1.756474,0.398217,00:30
1,1.753635,1.644160,0.454268,00:30
2,1.587111,1.526043,0.507261,00:30
3,1.502750,1.355749,0.558217,00:30
4,1.345973,1.378126,0.550828,00:30
5,1.273336,1.222539,0.590828,00:30
6,1.213820,1.100863,0.631847,00:30
7,1.129219,1.065984,0.654013,00:29
8,1.058663,1.046811,0.656560,00:30
9,1.008756,0.994588,0.678981,00:29
